In [2]:
import numpy as np
import pandas as pd
import pickle
import matplotlib as plt
from itertools import combinations
import re
import gseapy
from tqdm import tqdm
import gzip
import numpy as np
from multiprocessing import  Pool,cpu_count
from itertools import product
from functools import partial
from tqdm import tqdm
from time import sleep

In [3]:
drug_genepath = pd.read_csv('./AD_project/chemical-gene interactions.csv')

all_drug_table = pd.DataFrame(drug_genepath[['Chemical Name', 'Chemical ID']]).drop_duplicates().reset_index()
all_drug_table = all_drug_table.drop(columns=['index'])
all_drug_table['gene'] = ''
all_drug_table['num_gene'] = ''
all_drug_table['enrich_kegg'] = ''
all_drug_table['enrich_reactome'] = ''
all_drug_table['enrich_wiki'] = ''
chunk_size = 600
all_drug_table = np.array_split(all_drug_table, chunk_size)

FileNotFoundError: [Errno 2] No such file or directory: './AD_project/chemical-gene interactions.csv'

In [89]:
# Query KEGG
enrich_kegg = pd.DataFrame(columns = all_drug_table[0].columns)
for i in tqdm(range(600)):
    drug_table = all_drug_table[i]
    for x in drug_table['Chemical Name'].tolist():
        drug_table['gene'][drug_table['Chemical Name']==x] = str(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique().tolist())
        drug_table['num_gene'][drug_table['Chemical Name']==x] = len(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique())

        genelist = pd.DataFrame(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique().tolist())
        try:
            enr = gseapy.enrichr(gene_list=genelist,
                         gene_sets=['KEGG_2016'],
                         organism='Human',
                         cutoff=0.5
                        )
            table = pd.DataFrame(enr.results)
            table = table[table['Adjusted P-value']<0.05]
            drug_table['enrich_kegg'][drug_table['Chemical Name']==x] = str(table['Term'].tolist())
        except:
            continue

    enrich_kegg = pd.concat([enrich_kegg, drug_table])
#enrich_kegg.to_csv('./AD_project/Drug-pathway-GSEA/enrich_kegg.csv')

100%|██████████| 600/600 [2:45:00<00:00, 16.50s/it]  


,Chemical Name,Chemical ID,gene,num_gene,enrich_kegg,enrich_reactome,enrich_wiki
0,Copper,D003300,"['143B1', 'A1BG', 'A2M', 'A3GALT2', 'A4GALT', ...",5417,['PI3K-Akt signaling pathway Homo sapiens hsa0...,"['Metabolism Homo sapiens R-HSA-1430728', 'Hem...",
1,Cadmium,D002104,"['18W', 'A1BG', 'A2M', 'AAA1', 'AAMP', 'ABAT',...",2841,"['Pathways in cancer Homo sapiens hsa05200', '...","['Metabolism Homo sapiens R-HSA-1430728', 'Hem...",
2,Dichlorodiphenyl Dichloroethylene,D003633,"['1-SF', 'AB124611', 'ABCB1', 'ABCB11', 'ABCB1...",619,"['Apoptosis Homo sapiens hsa04210', 'Pathways ...",['Developmental Biology Homo sapiens R-HSA-126...,
3,cupric chloride,C029892,"['3101707', 'AASDHPPT', 'ABCB8', 'ABCF2', 'ABC...",869,"['Apoptosis Homo sapiens hsa04210', 'Hepatitis...","['Metabolism Homo sapiens R-HSA-1430728', 'Pro...",
4,Folic Acid,D005492,"['A', 'A4GALT', 'AAAS', 'AACS', 'AADAC', 'AAGA...",10068,"['Pathways in cancer Homo sapiens hsa05200', '...","['Metabolism Homo sapiens R-HSA-1430728', 'Met...",
...,...,...,...,...,...,...,...
6543,Tetanus Toxin,D013744,['VEGFA'],1,,,
6544,tetraethylenepentamine,C034269,['VEGFA'],1,,['VEGF ligand-receptor interactions Homo sapie...,
6545,urushiol,C003747,['VEGFA'],1,,,
6546,"vasoactive intestinal peptide, 4-chloro-Phe(6)...",C048507,['VEGFA'],1,,['VEGF ligand-receptor interactions Homo sapie...,


In [63]:
# Query Reactome
enrich_reactome = pd.DataFrame(columns = all_drug_table[0].columns)
for i in tqdm(range(600)):
    drug_table = all_drug_table[i]
    for x in drug_table['Chemical Name'].tolist():
        drug_table['gene'][drug_table['Chemical Name']==x] = str(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique().tolist())
        drug_table['num_gene'][drug_table['Chemical Name']==x] = len(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique())

        genelist = pd.DataFrame(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique().tolist())
        try:
            enr = gseapy.enrichr(gene_list=genelist,
                         gene_sets=['Reactome_2016'],
                         organism='Human',
                         cutoff=0.5
                        )
            table = pd.DataFrame(enr.results)
            table = table[table['Adjusted P-value']<0.05]
            drug_table['enrich_reactome'][drug_table['Chemical Name']==x] = str(table['Term'].tolist())
        except:
            continue
    enrich_reactome = pd.concat([enrich_reactome, drug_table])
#enrich_reactome.to_csv('./AD_project/Drug-pathway-GSEA/enrich_reactome.csv')

100%|██████████| 600/600 [2:50:14<00:00, 17.02s/it]  


,Chemical Name,Chemical ID,gene,num_gene,enrich_kegg,enrich_reactome,enrich_wiki
0,Copper,D003300,"['143B1', 'A1BG', 'A2M', 'A3GALT2', 'A4GALT', ...",5417,,"['Metabolism Homo sapiens R-HSA-1430728', 'Hem...",
1,Cadmium,D002104,"['18W', 'A1BG', 'A2M', 'AAA1', 'AAMP', 'ABAT',...",2841,,"['Metabolism Homo sapiens R-HSA-1430728', 'Hem...",
2,Dichlorodiphenyl Dichloroethylene,D003633,"['1-SF', 'AB124611', 'ABCB1', 'ABCB11', 'ABCB1...",619,,['Developmental Biology Homo sapiens R-HSA-126...,
3,cupric chloride,C029892,"['3101707', 'AASDHPPT', 'ABCB8', 'ABCF2', 'ABC...",869,,"['Metabolism Homo sapiens R-HSA-1430728', 'Pro...",
4,Folic Acid,D005492,"['A', 'A4GALT', 'AAAS', 'AACS', 'AADAC', 'AAGA...",10068,,"['Metabolism Homo sapiens R-HSA-1430728', 'Met...",
...,...,...,...,...,...,...,...
6543,Tetanus Toxin,D013744,['VEGFA'],1,,,
6544,tetraethylenepentamine,C034269,['VEGFA'],1,,['VEGF ligand-receptor interactions Homo sapie...,
6545,urushiol,C003747,['VEGFA'],1,,,
6546,"vasoactive intestinal peptide, 4-chloro-Phe(6)...",C048507,['VEGFA'],1,,['VEGF ligand-receptor interactions Homo sapie...,


In [94]:
# Query Wiki
enrich_wiki = pd.DataFrame(columns = all_drug_table[0].columns)
for i in tqdm(range(600)):
    drug_table = all_drug_table[i]
    for x in drug_table['Chemical Name'].tolist():
        drug_table['gene'][drug_table['Chemical Name']==x] = str(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique().tolist())
        drug_table['num_gene'][drug_table['Chemical Name']==x] = len(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique())

        genelist = pd.DataFrame(drug_genepath[drug_genepath['Chemical Name'] == x]['Gene Symbol'].unique().tolist())
        try:
            enr = gseapy.enrichr(gene_list=genelist,
                         gene_sets=['WikiPathways_2019_Human'],
                         organism='Human',
                         cutoff=0.5
                        )
            table = pd.DataFrame(enr.results)
            table = table[table['Adjusted P-value']<0.05]
            drug_table['enrich_wiki'][drug_table['Chemical Name']==x] = str(table['Term'].tolist())
        except:
            continue

    enrich_wiki = pd.concat([enrich_wiki, drug_table])
#enrich_wiki.to_csv('./AD_project/Drug-pathway-GSEA/enrich_wiki.csv')

 20%|█▉        | 118/600 [45:23<2:33:04, 19.05s/it]2021-09-02 21:59:50,901 Error fetching enrichment results: WikiPathways_2019_Human
2021-09-02 21:59:50,991 Error fetching enrichment results: WikiPathways_2019_Human
 33%|███▎      | 198/600 [1:16:19<2:49:36, 25.32s/it]2021-09-02 22:31:02,831 Error fetching enrichment results: WikiPathways_2019_Human
2021-09-02 22:31:02,917 Error fetching enrichment results: WikiPathways_2019_Human
 58%|█████▊    | 350/600 [1:57:33<51:10, 12.28s/it]  2021-09-02 23:12:04,064 Error fetching enrichment results: WikiPathways_2019_Human
2021-09-02 23:12:04,157 Error fetching enrichment results: WikiPathways_2019_Human
100%|██████████| 600/600 [2:35:17<00:00, 15.53s/it]


,Chemical Name,Chemical ID,gene,num_gene,enrich_kegg,enrich_reactome,enrich_wiki
0,Copper,D003300,"['143B1', 'A1BG', 'A2M', 'A3GALT2', 'A4GALT', ...",5417,['PI3K-Akt signaling pathway Homo sapiens hsa0...,"['Metabolism Homo sapiens R-HSA-1430728', 'Hem...","['PI3K-Akt Signaling Pathway WP4172', 'Focal A..."
1,Cadmium,D002104,"['18W', 'A1BG', 'A2M', 'AAA1', 'AAMP', 'ABAT',...",2841,"['Pathways in cancer Homo sapiens hsa05200', '...","['Metabolism Homo sapiens R-HSA-1430728', 'Hem...","['Nuclear Receptors Meta-Pathway WP2882', 'VEG..."
2,Dichlorodiphenyl Dichloroethylene,D003633,"['1-SF', 'AB124611', 'ABCB1', 'ABCB11', 'ABCB1...",619,"['Apoptosis Homo sapiens hsa04210', 'Pathways ...",['Developmental Biology Homo sapiens R-HSA-126...,"['Nuclear Receptors Meta-Pathway WP2882', 'Apo..."
3,cupric chloride,C029892,"['3101707', 'AASDHPPT', 'ABCB8', 'ABCF2', 'ABC...",869,"['Apoptosis Homo sapiens hsa04210', 'Hepatitis...","['Metabolism Homo sapiens R-HSA-1430728', 'Pro...","['Copper homeostasis WP3286', 'Hepatitis C and..."
4,Folic Acid,D005492,"['A', 'A4GALT', 'AAAS', 'AACS', 'AADAC', 'AAGA...",10068,"['Pathways in cancer Homo sapiens hsa05200', '...","['Metabolism Homo sapiens R-HSA-1430728', 'Met...","['TGF-beta Signaling Pathway WP366', 'VEGFA-VE..."
...,...,...,...,...,...,...,...
6543,Tetanus Toxin,D013744,['VEGFA'],1,,,
6544,tetraethylenepentamine,C034269,['VEGFA'],1,,['VEGF ligand-receptor interactions Homo sapie...,
6545,urushiol,C003747,['VEGFA'],1,,,
6546,"vasoactive intestinal peptide, 4-chloro-Phe(6)...",C048507,['VEGFA'],1,,['VEGF ligand-receptor interactions Homo sapie...,
